### Import packages

In [1]:
import pandas as pd
import numpy as np

import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### load dataset

In [2]:
df_raw = pd.read_csv("C:/Users/user/RSpirit/winemag-data_first150k.csv")

In [3]:
df_raw.head()

Unnamed: 0 country                                        description  \
0           0      US  This tremendous 100% varietal wine hails from ...   
1           1   Spain  Ripe aromas of fig, blackberry and cassis are ...   
2           2      US  Mac Watson honors the memory of a wine once ma...   
3           3      US  This spent 20 months in 30% new French oak, an...   
4           4  France  This is the top wine from La Bégude, named aft...   

                            designation  points  price        province  \
0                     Martha's Vineyard      96  235.0      California   
1  Carodorum Selección Especial Reserva      96  110.0  Northern Spain   
2         Special Selected Late Harvest      96   90.0      California   
3                               Reserve      96   65.0          Oregon   
4                            La Brûlade      95   66.0        Provence   

            region_1           region_2             variety  \
0        Napa Valley               Napa  Cabernet Sauvignon   
1               Toro                NaN       Tinta de Toro   
2     Knights Valley             Sonoma     Sauvignon Blanc   
3  Willamette Valley  Willamette Valley          Pinot Noir   
4             Bandol                NaN  Provence red blend   

                    winery  
0                    Heitz  
1  Bodega Carmen Rodríguez  
2                 Macauley  
3                    Ponzi  
4     Domaine de la Bégude

In [4]:
df = pd.DataFrame(df_raw[['description', 'points']])

In [5]:
df.head()

description  points
0  This tremendous 100% varietal wine hails from ...      96
1  Ripe aromas of fig, blackberry and cassis are ...      96
2  Mac Watson honors the memory of a wine once ma...      96
3  This spent 20 months in 30% new French oak, an...      96
4  This is the top wine from La Bégude, named aft...      95

### rank estimator

In [7]:
def rank_estimator(score):
    
    if (85 >= score >= 80):
        return "B"
    elif (90 >= score > 85):
        return "A"
    elif (95 >= score > 90):
        return "S"
    else:
        return "SS"

df_revised = df.copy()
df_revised.points = df_revised.points.apply(rank_estimator)
df_revised.head()

description points
0  This tremendous 100% varietal wine hails from ...     SS
1  Ripe aromas of fig, blackberry and cassis are ...     SS
2  Mac Watson honors the memory of a wine once ma...     SS
3  This spent 20 months in 30% new French oak, an...     SS
4  This is the top wine from La Bégude, named aft...      S

In [15]:
y = df_revised['points']

### Re preprocessing

In [9]:
description = df_revised['description']

In [10]:
description = description.apply(lambda x: x.lower())

description = description.apply(lambda x: re.sub('[^\w\s]', ' ', x))
description = description.apply(lambda x: re.sub('\d', ' ', x))
description = description.apply(lambda x: re.sub('[ \t]+' , ' ', x))

In [12]:
description[0]

'this tremendous varietal wine hails from oakville and was aged over three years in oak juicy red cherry fruit and a compelling hint of caramel greet the palate framed by elegant fine tannins and a subtle minty tone in the background balanced and rewarding from start to finish it has years ahead of it to develop further nuance enjoy '

### CountVectorizer

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english', tokenizer=word_tokenize)

corpus = cv.fit_transform(description)

max_df is used for removing terms that appear too frequently, also known as "corpus-specific stop words".  

For example:  
  
max_df = 0.50 means "ignore terms that appear in more than 50% of the documents".  
max_df = 25 means "ignore terms that appear in more than 25 documents".  

In [30]:
corpus.shape

(150930, 29859)

### Multinomial NB

In [31]:
from sklearn.naive_bayes import MultinomialNB

multi_nb = MultinomialNB()

multi_nb.fit(corpus, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

### Cross-Validation

In [32]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(multi_nb, corpus, y, cv=5)

In [33]:
scores

array([0.67194488, 0.69688939, 0.70661896, 0.72632102, 0.7156866 ])

### Train-Test split

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.3, random_state=42)

In [37]:
multi_nb.fit(X_train, y_train)

y_pred = multi_nb.predict(X_test)

print((y_pred == y_test).mean())

0.7048742242540692
